In [ ]:
from pyspark.sql.functions import *
import matplotlib.pyplot as plt
import matplotlib
from pyspark.ml.regression import LinearRegression
from pyspark.ml.feature import VectorAssembler
from pyspark.sql.window import *
from IPython.core.display import HTML
import numpy as np

display(HTML("<style>pre { white-space: pre !important; } video { width: 100% }</style>"))
matplotlib.use("pgf")
matplotlib.rcParams.update({
    "pgf.texsystem": "pdflatex",
    'font.family': 'serif',
    'text.usetex': True,
    'pgf.rcfonts': False,
})
sc.setLogLevel("ERROR")

In [ ]:
df = spark.read.csv("/user/s1919377/flights/*", header='true')
df = df.withColumn("firstseen",to_timestamp("firstseen", "yyyy-MM-dd HH:mm:ss")) \
       .withColumn("lastseen",to_timestamp("lastseen", "yyyy-MM-dd HH:mm:ss")) \
       .withColumn("day",to_timestamp("day", "yyyy-MM-dd HH:mm:ss")) \
       .withColumn("longitude_1",col("longitude_1").cast("float")) \
       .withColumn("longitude_2",col("longitude_2").cast("float")) \
       .withColumn("latitude_1",col("latitude_1").cast("float")) \
       .withColumn("latitude_2",col("latitude_2").cast("float")) \
       .withColumn("altitude_1",col("altitude_1").cast("float")) \
       .withColumn("altitude_2",col("altitude_2").cast("float"))
df.show(truncate=False)
df.printSchema()

In [ ]:
airport_metadata = spark.read.json('file:///home/s1919377/project/airport_codes.json') \
    .withColumn("longitude",split("coordinates", ', ').getItem(0).cast("float")) \
    .withColumn("latitude",split("coordinates", ', ').getItem(1).cast("float"))

In [ ]:
df_first_month = df.where((month("day") == 1) & (year("day") == 2019))

In [ ]:
large_airports = airport_metadata.where(col("type") == "large_airport")
airports_by_destination_count = df_first_month \
    .where(col("destination").isNotNull()) \
    .groupby("destination") \
    .count() \
    .join(large_airports, [large_airports.ident == col("destination")], "leftsemi")

top_100_airports = airports_by_destination_count \
    .sort(col("count").desc()) \
    .limit(100)

In [ ]:
df_first_month = df \
    .where((month("day") == 1) & (year("day") == 2019)) \
    .join(top_100_airports, "destination", "leftsemi")
df_last_month = df \
    .where((month("day") == 12) & (year("day") == 2021)) \
    .join(top_100_airports, "destination", "leftsemi")
df_mid_month = df \
    .where((month("day") == 4) & (year("day") == 2020)) \
    .join(top_100_airports, "destination", "leftsemi")

In [ ]:
df_first_month_counts = df_first_month \
    .groupby("destination") \
    .count() \
    .withColumn("ix", dense_rank().over(Window.orderBy(col("count").desc()))) \
    .persist()
df_last_month_counts = df_last_month \
    .groupby("destination") \
    .count() \
    .withColumnRenamed("count", "count_temp") \
    .join(df_first_month_counts, "destination", "left") \
    .drop("count") \
    .withColumnRenamed("count_temp", "count") \
    .sort(col("ix").asc()) \
    .persist()
df_mid_month_counts = df_mid_month \
    .groupby("destination") \
    .count() \
    .withColumnRenamed("count", "count_temp") \
    .join(df_first_month_counts, "destination", "left") \
    .drop("count") \
    .withColumnRenamed("count_temp", "count") \
    .sort(col("ix").asc()) \
    .persist()

In [ ]:
vectorAssembler = VectorAssembler(inputCols = ['ix'], outputCol = 'features')

vfirst_df = vectorAssembler.transform(df_first_month_counts)
vfirst_df = vfirst_df.select(['features', 'count'])

lr_first = LinearRegression(featuresCol='features', labelCol='count', maxIter=10, regParam=0.3, elasticNetParam=0.8)
lr_model_first = lr_first.fit(vfirst_df)
print("Coefficients: " + str(lr_model_first.coefficients))
print("Intercept: " + str(lr_model_first.intercept))
trainingSummary_first = lr_model_first.summary
print("RMSE: %f" % trainingSummary_first.rootMeanSquaredError)
print("r2: %f" % trainingSummary_first.r2)

In [ ]:
vlast_df = vectorAssembler.transform(df_last_month_counts)
vlast_df = vlast_df.select(['features', 'count'])

lr_last = LinearRegression(featuresCol='features', labelCol='count', maxIter=10, regParam=0.3, elasticNetParam=0.8)
lr_model_last = lr_last.fit(vlast_df)
print("Coefficients: " + str(lr_model_last.coefficients))
print("Intercept: " + str(lr_model_last.intercept))
trainingSummary_last = lr_model_last.summary
print("RMSE: %f" % trainingSummary_last.rootMeanSquaredError)
print("r2: %f" % trainingSummary_last.r2)

In [ ]:
vmid_df = vectorAssembler.transform(df_mid_month_counts)
vmid_df = vmid_df.select(['features', 'count'])

lr_mid = LinearRegression(featuresCol='features', labelCol='count', maxIter=10, regParam=0.3, elasticNetParam=0.8)
lr_mid_last = lr_mid.fit(vmid_df)
print("Coefficients: " + str(lr_mid_last.coefficients))
print("Intercept: " + str(lr_mid_last.intercept))
trainingSummary_mid = lr_mid_last.summary
print("RMSE: %f" % trainingSummary_mid.rootMeanSquaredError)
print("r2: %f" % trainingSummary_mid.r2)

In [ ]:
first_vals = np.array(df_first_month_counts.select("ix", "count").collect())
plt.plot(first_vals[:, 0], first_vals[:, 1], label="January 2019")
plt.plot(first_vals[:, 0], first_vals[:, 0] * lr_model_first.coefficients[0] + lr_model_first.intercept, linestyle="dashed")
plt.title("Top 100 Airports in January 2019")
plt.savefig('dist_first.pgf')

In [ ]:
last_vals = np.array(df_last_month_counts.select("ix", "count").collect())
plt.plot(last_vals[:, 0], last_vals[:, 1], label="December 2021")
plt.plot(last_vals[:, 0], last_vals[:, 0] * lr_model_last.coefficients[0] + lr_model_last.intercept, linestyle="dashed")
plt.title("Top 100 Airports in December 2021")
plt.savefig('dist_last.pgf')

In [ ]:
mid_vals = np.array(df_mid_month_counts.select("ix", "count").collect())
plt.plot(mid_vals[:, 0], mid_vals[:, 1], label="December 2021")
plt.plot(mid_vals[:, 0], mid_vals[:, 0] * lr_mid_last.coefficients[0] + lr_mid_last.intercept, linestyle="dashed")
plt.title("Top 100 Airports in April 2021")
plt.savefig('dist_mid.pgf')